In [ ]:
from datetime import datetime

import numpy as np
import html
import re
import neattext as nt
import neattext.functions as nfx
import nltk
nltk.data.path.append("/dbfs/tmp/nltk")
nltk.download('stopwords', download_dir="/dbfs/tmp/nltk")
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import torch

from transformers import pipeline
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover

[nltk_data] Downloading package stopwords to /dbfs/tmp/nltk...
[nltk_data] Package stopwords is already up-to-date!

In [ ]:
spark = SparkSession.builder.getOrCreate()

## Loading Data from MongoDB

### Posts Dataframe

In [ ]:
database = 'reddit_project'
user_name = 'admin'
password = 'msds697'
address = 'msds697-cluster.c0ekx.mongodb.net'

In [ ]:
collection_posts = 'posts'
connection_string_posts = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection_posts}"

In [ ]:
df_posts = spark.read.format("mongo").option("uri",connection_string_posts).load()
df_posts.show(10)

+--------------------+----+--------------------+-------+------------+-----+--------------------+--------------------+--------------------+
 _id|body| created| id|num_comments|score| subreddit| title| url|
+--------------------+----+--------------------+-------+------------+-----+--------------------+--------------------+--------------------+
[63f9391c4c432970...|null|2023-02-24 10:49:...|11aytdc| 422|13517| wholesomememes|It feels nice to ...|https://i.imgur.c...|
[63f9391d4c432970...|null|2023-02-24 10:17:...|11ay1b8| 1084|13212| mildlyinfuriating|Wearing this shir...|https://i.imgur.c...|
[63f9391e4c432970...|null|2023-02-24 09:07:...|11awd4i| 1627|30474| MadeMeSmile|9 Year Old Recent...|https://i.redd.it...|
[63f939204c432970...|null|2023-02-24 09:22:...|11awq54| 2008|18370| ContagiousLaughter| Best answer|https://v.redd.it...|
[63f939214c432970...|null|2023-02-24 07:25:...|11atymc| 708|28451| todayilearned|TIL that dogs int...|https://www.petmd...|
[63f939214c432970...|null|2023-02-24 09:53:...|11axgd4| 1573| 9745|Damnthatsinteresting|I always have the...|https://i.redd.it...|
[63f939214c432970...|null|2023-02-24 09:44:...|11ax9g0| 457| 8439| IASIP|Looks like we’re ...|https://i.redd.it...|
[63f939214c432970...|null|2023-02-24 09:56:...|11axj63| 744| 9001| interestingasfuck|Man dies without ...|https://www.reddi...|
[63f939214c432970...|null|2023-02-24 07:11:...|11atnkt| 248|22158| funny|I’m not normally ...|https://i.redd.it...|
[63f939224c432970...|null|2023-02-24 06:54:...|11at9gp| 498|31225| oddlysatisfying|A Collage Of All ...|https://i.redd.it...|
+--------------------+----+--------------------+-------+------------+-----+--------------------+--------------------+--------------------+
only showing top 10 rows

### Comments Dataframe

In [ ]:
collection_comments = 'comments'
connection_string_comments = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection_comments}"

In [ ]:
df_comments = spark.read.format("mongo").option("uri",connection_string_comments).load()
df_comments.rdd.count()

Out[25]: 39160

In [ ]:
df_comments.show(5)

+--------------------+--------------------+-------+
 _id| comment|post_id|
+--------------------+--------------------+-------+
[63f92e214c432970...|If u can get to 1...|11aytdc|
[63f92e224c432970...|I'm at day 13 now...|11aytdc|
[63f92e224c432970...|Quit for 54 days ...|11aytdc|
[63f92e224c432970...|Get it! I’m at 36...|11aytdc|
[63f92e224c432970...|I'm on day 7. Con...|11aytdc|
+--------------------+--------------------+-------+
only showing top 5 rows

### Kaggle Dataframe

In [ ]:
collection_kaggle = 'kaggle'
connection_string_kaggle = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection_kaggle}"

In [ ]:
df_kaggle = spark.read.format("mongo").option("uri",connection_string_kaggle).load()
df_kaggle.rdd.count()

Out[32]: 26402

In [ ]:
df_kaggle.show(5)

+---+--------------------+-----+-----+-----------+-------+------------------+--------------------+
_c0| _id|class|count|hate_speech|neither|offensive_language| tweet|
+---+--------------------+-----+-----+-----------+-------+------------------+--------------------+
 0|[63f90dfeb6ced0f4...| 2| 3| 0| 3| 0|!!! RT @mayasolov...|
 1|[63f90dfeb6ced0f4...| 1| 3| 0| 0| 3|!!!!! RT @mleew17...|
 2|[63f90dffb6ced0f4...| 1| 3| 0| 0| 3|!!!!!!! RT @UrKin...|
 3|[63f90e00b6ced0f4...| 1| 3| 0| 1| 2|!!!!!!!!! RT @C_G...|
 4|[63f90e00b6ced0f4...| 1| 6| 0| 0| 6|!!!!!!!!!!!!! RT ...|
+---+--------------------+-----+-----+-----------+-------+------------------+--------------------+
only showing top 5 rows

In [ ]:
df_kaggle.printSchema()

root
-- _c0: string (nullable = true)
-- _id: struct (nullable = true)
 |-- oid: string (nullable = true)
-- class: string (nullable = true)
-- count: string (nullable = true)
-- hate_speech: string (nullable = true)
-- neither: string (nullable = true)
-- offensive_language: string (nullable = true)
-- tweet: string (nullable = true)

## Data Cleaning

Goals: 
1. Drop null values in tweets
2. Tokenize the tweets and then perform word embedding on tokens by word counts
3. Split the three-class label column into two binary-class label columns

In [ ]:
df = df_kaggle.filter(df_kaggle["tweet"].isNotNull())\
    .withColumn("hate_speech", when(df_kaggle["class"] == '1', 1).otherwise(0))\
    .withColumn("offensive_language", when(df_kaggle["class"] == '2', 1).otherwise(0))\
    .select("tweet", "hate_speech", "offensive_language")

df.show()

+--------------------+-----------+------------------+
 tweet|hate_speech|offensive_language|
+--------------------+-----------+------------------+
!!! RT @mayasolov...| 0| 1|
!!!!! RT @mleew17...| 1| 0|
!!!!!!! RT @UrKin...| 1| 0|
!!!!!!!!! RT @C_G...| 1| 0|
!!!!!!!!!!!!! RT ...| 1| 0|
"!!!!!!!!!!!!!!!!...| 1| 0|
"!!!!!!""@__Brigh...| 1| 0|
!!!!&#8220;@selfi...| 1| 0|
""" &amp; you mig...| 1| 0|
""" @rhythmixx_ :...| 1| 0|
""" Keeks is a bi...| 1| 0|
""" Murda Gang bi...| 1| 0|
""" So hoes that ...| 1| 0|
""" bad bitches i...| 1| 0|
""" bitch get up ...| 1| 0|
""" bitch nigga m...| 1| 0|
""" bitch plz wha...| 1| 0|
""" bitch who do ...| 1| 0|
""" bitches get c...| 1| 0|
""" black bottle ...| 1| 0|
+--------------------+-----------+------------------+
only showing top 20 rows

In [ ]:
hs_data = df.select("tweet", "hate_speech")

coltokenizer = RegexTokenizer(inputCol='tweet', outputCol='words', pattern='\\W')
hs_data = coltokenizer.transform(hs_data)

stopwords_remover = StopWordsRemover(inputCol='words', outputCol='filtered_words')
hs_data = stopwords_remover.transform(hs_data)

count_vectorizer = CountVectorizer(inputCol='filtered_words', outputCol='features')
hs_data = count_vectorizer.fit(hs_data).transform(hs_data)
hs_data.show()

+--------------------+-----------+--------------------+--------------------+--------------------+
 tweet|hate_speech| words| filtered_words| features|
+--------------------+-----------+--------------------+--------------------+--------------------+
!!! RT @mayasolov...| 0|[rt, mayasolovely...|[rt, mayasolovely...|(34913,[1,20,23,3...|
!!!!! RT @mleew17...| 1|[rt, mleew17, boy...|[rt, mleew17, boy...|(34913,[1,9,35,97...|
!!!!!!! RT @UrKin...| 1|[rt, urkindofbran...|[rt, urkindofbran...|(34913,[0,1,15,18...|
!!!!!!!!! RT @C_G...| 1|[rt, c_g_anderson...|[rt, c_g_anderson...|(34913,[1,5,40,63...|
!!!!!!!!!!!!! RT ...| 1|[rt, shenikarober...|[rt, shenikarober...|(34913,[0,1,18,39...|
"!!!!!!!!!!!!!!!!...| 1|[t_madison_x, the...|[t_madison_x, shi...|(34913,[2,7,18,30...|
"!!!!!!""@__Brigh...| 1|[__brighterdays, ...|[__brighterdays, ...|(34913,[0,16,18,4...|
!!!!&#8220;@selfi...| 1|[8220, selfiequee...|[8220, selfiequee...|(34913,[3,10,11,1...|
""" &amp; you mig...| 1|[amp, you, might,...|[amp, might, get,...|(34913,[0,14,23,3...|
""" @rhythmixx_ :...| 1|[rhythmixx_, hobb...|[rhythmixx_, hobb...|(34913,[1289,3500...|
""" Keeks is a bi...| 1|[keeks, is, a, bi...|[keeks, bitch, cu...|(34913,[0,5,21,14...|
""" Murda Gang bi...| 1|[murda, gang, bit...|[murda, gang, bit...|(34913,[0,1322,20...|
""" So hoes that ...| 1|[so, hoes, that, ...|[hoes, smoke, los...|(34913,[7,31,352,...|
""" bad bitches i...| 1|[bad, bitches, is...|[bad, bitches, th...|(34913,[3,5,35,16...|
""" bitch get up ...| 1|[bitch, get, up, ...| [bitch, get]|(34913,[0,14],[1....|
""" bitch nigga m...| 1|[bitch, nigga, mi...|[bitch, nigga, miss]|(34913,[0,19,425]...|
""" bitch plz wha...| 1|[bitch, plz, what...|[bitch, plz, what...|(34913,[0,656,213...|
""" bitch who do ...| 1|[bitch, who, do, ...| [bitch, love]|(34913,[0,28],[1....|
""" bitches get c...| 1|[bitches, get, cu...|[bitches, get, cu...|(34913,[3,14,189,...|
""" black bottle ...| 1|[black, bottle, a...|[black, bottle, a...|(34913,[0,23,35,1...|
+--------------------+-----------+--------------------+--------------------+--------------------+
only showing top 20 rows

## Model Training and Evaluation

### Naive Bayes Classifier

Split data into training and test data. For each of the "hate speech" and "offensive language" label columns, fit training data with a Naive Bayes Classifier and predict on test data. Record the f1 scores of prediction against true labels of test data.

In [ ]:
#Split training and Testing
train_val_hs, test_hs = hs_data.randomSplit([0.8, 0.2], seed=42)
train_hs, val_hs = train_val_hs.randomSplit([0.8, 0.2], seed=42)

In [ ]:
naive_bayes_hs = NaiveBayes(smoothing=1.0, modelType='multinomial', labelCol='hate_speech')
hs_model = naive_bayes_hs.fit(train_hs)
hs_pred = hs_model.transform(test_hs)

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='hate_speech', metricName='f1')
hs_f1_score = evaluator.evaluate(hs_pred)
hs_f1_score

Out[38]: 0.8707424601007747

In [ ]:
ol_data = df.select("tweet", "offensive_language")

tokenizer = RegexTokenizer(inputCol='tweet', outputCol='words', pattern='\\W')
ol_data = tokenizer.transform(ol_data)

stopwords_remover = StopWordsRemover(inputCol='words', outputCol='filtered_words')
ol_data = stopwords_remover.transform(ol_data)

count_vectorizer = CountVectorizer(inputCol='filtered_words', outputCol='features')
ol_data = count_vectorizer.fit(ol_data).transform(ol_data)

#Split training and Testing
train_val_ol, test_ol = ol_data.randomSplit([0.8, 0.2], seed=42)
train_ol, val_ol = train_val_ol.randomSplit([0.8, 0.2], seed=42)

naive_bayes_ol = NaiveBayes(smoothing=1.0, modelType='multinomial', labelCol='offensive_language')
ol_model = naive_bayes_ol.fit(train_ol)
ol_pred = ol_model.transform(test_ol)

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='offensive_language', metricName='f1')
ol_f1_score = evaluator.evaluate(ol_pred)
ol_f1_score

Out[39]: 0.916463365861462

The f1 score for detecting hate speech in tweets is 0.8707 while the f1 score for detecting offensive language in tweets is 0.9164.

## Next Steps

Try out the Naive Bayes Classifier on the reddit comment data to see how it performs.